### 도구 호출 에이전트(Tool Calling Agent)

In [33]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)
llm.invoke([HumanMessage("부산은 지금 몇시야?")])

AIMessage(content='현재 시간으로 알려드릴게요.\n\n지금 부산은 **대한민국 표준시(KST)**를 사용하며, 이는 협정 세계시(UTC)보다 9시간 빠릅니다.\n\n**2024년 5월 15일 17시 00분 (수요일)** 기준으로 말씀드리면,\n\n**현재 부산은 2024년 5월 15일 오후 5시 00분 입니다.**\n\n(제가 답변을 드리는 시점에 따라 실제 시간은 몇 분 오차가 있을 수 있습니다.)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--ff5e6879-18aa-4de5-a113-611bd84abf7b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 1062, 'total_tokens': 1070, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 942}})

In [ ]:
from langchain_core.tools import tool
from datetime import datetime
from zoneinfo import ZoneInfo

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    target_timezone = ZoneInfo(timezone)
    now = datetime.now(target_timezone).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [37]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# 도구들을 tools 리스트에 추가
tools = [get_current_time,]

# 에이전트 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 초기화
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
from langchain_core.messages import SystemMessage

# 메시지 형식 변경 (agent_executor에 맞게)
messages = [
    HumanMessage("부산은 지금 몇시야?"),
]

# agent_executor를 사용하여 실행
response = agent_executor.invoke({"input": messages})



> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{'timezone': 'Asia/Seoul', 'location': '부산'}`


Asia/Seoul (부산) 현재시각 2025-08-07 17:55:22 
Asia/Seoul (부산) 현재시각 2025-08-07 17:55:22 부산은 지금 2025년 8월 7일 17시 55분 22초입니다.


> Finished chain.


In [39]:
response


{'input': [HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={})],
 'output': '부산은 지금 2025년 8월 7일 17시 55분 22초입니다.\n'}